## Getting ASEC data from CPS API


In [ ]:
import pandas as pd
import numpy as np
import pathlib as pl
import os
import requests

ASEC_VARIABLES = [
    'PEMLR', # labor force status
    'WKSWORK', # weeks worked last year
    'A_AGE',
    'A_SEX',
    'A_HGA',
    'PRDTRACE',
    'PEHSPNON',
    'PRDISFLG',
    'PRCITSHP',
    'MARSUPWT',
    'A_LFSR',
    'HRCHECK',
    'A_CLSWKR',
    'CLWK',
    'A_DTOCC', # last year major occ
    'WEMIND', # last year major ind
    'A_MJIND',
    'LJCW',
    'HTOTVAL',
    'A_MARITL',
    'FPERSONS',  # number of persons in family
    'FRELU18',  # number of persons in family under 18
]

# this function will retrieve an entire year of ASEC data
def get_asec_year_df(api_key, asec_variables, year):
    base_url = f'https://api.census.gov/data/{year}/cps/asec/mar'
    get_vars = ','.join(asec_variables)
    url = f'{base_url}?get={get_vars}&key={api_key}'
    response = requests.get(url)
    
    data = response.json()
    df = pd.DataFrame(data[1:], columns=data[0])
    
    df = df.rename(columns={x: x.lower() for x in df.columns})
    df.loc[:, 'year'] = year
    
    for var in df.columns:
        if var == 'marsupwt':
            df[var] = df[var].astype(np.float64)
        else:
            df[var] = df[var].astype(np.int64)
            
    return df

# I've saved my API key in a local file
with open('c:/users/calvi/coding/census_api_key.txt', 'r') as f:
    api_key = f.readline().strip()

asec = pd.DataFrame()
base_year = 2024
start_year = base_year - 9
for year in range(start_year, base_year + 1):
    print(f'Processing year {year}')
    df_year = get_asec_year_df(api_key, ASEC_VARIABLES, year)
    asec = pd.concat([
        asec,
        df_year
    ])

asec.head()

## Transform data to prepare for modeling

### Get price data from BLS

We need to account for inflation to make incomes comparable across years.


In [ ]:
import json

BLS_CPI_SERIES = 'CUUR0000SA0'  # CPI-U

# Get CPI data from BLS API 
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": [BLS_CPI_SERIES],
                   "startyear": str(start_year - 1), 
                   "endyear": str(base_year-  1)})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', 
                  data=data, headers=headers)
json_data = json.loads(p.text)

# put CPI data into dataframe and calculate inflation rates
cpi = pd.DataFrame(json_data['Results']['series'][0]['data'])
cpi = cpi[cpi.period == 'M12']  # only keep end of year numbers
cpi = cpi[['year', 'value']]
cpi.year = cpi.year.astype(np.int64)
cpi.value = cpi.value.astype(np.float64)
cpi = cpi.sort_values('year', ascending=False).reset_index(drop=True)
cpi['inflation'] = cpi.value / cpi.value.shift(-1) - 1
cpi

Inflation numbers look accurate. Let's Cacluate price level ratios indexed to 2023 prices now. The years will be shifted by one, because incomes in each year of ASEC data correspond to the previous year's income:


In [ ]:
cpi_2023 = cpi[cpi.year == 2023].value.iloc[0]
cpi['pl_ratio'] = cpi_2023 / cpi.value
cpi['year'] = cpi['year'] + 1
cpi

### Clean ASEC data


In [ ]:
def remove_rows(df):
    df = df.copy()

    # only include people who worked in paying jobs last year
    df = df[df.ljcw.isin([1,2,3,4,5,6])].reset_index(drop=True)

    # only include people who worked more than 30 weeks
    df = df[df.wkswork > 30]

    return df

def get_edu_var(df):
    df = df.copy()
    bins = [30, 39, 40, 41, 43, 44, 45, np.inf]
    names = ['LTHS', 'HS', 'SCND', 'AD', 'BA', 'MA', 'DOC']
    df['edu'] = pd.cut(df['a_hga'], bins, labels=names, right=False)
    
    return df

def get_race_var(df):
    df = df.copy()
    df['race'] = 'other'
    df.loc[df.prdtrace == 1, 'race'] = 'white'
    df.loc[df.prdtrace == 2, 'race'] = 'black'
    df.loc[df.prdtrace == 4, 'race'] = 'asian'
    
    return df

def get_hispanic_var(df):
    df = df.copy()
    df['hisp'] = 0
    df.loc[df.pehspnon==1, 'hisp'] = 1
    
    return df

def get_male_var(df):
    df = df.copy()
    df['male'] = 0
    df.loc[df.a_sex==1, 'male'] = 1
    
    return df

def get_citizenship_var(df):
    df = df.copy()
    bins = [1, 4, 5, np.inf]
    names = ['native', 'naturalized', 'noncitizen']
    df.loc[:, 'citshp'] = pd.cut(df['prcitshp'],
                                 bins=bins, labels=names, right=False, include_lowest=True)
    
    return df

def get_cow_var(df):
    df = df.copy()
    bins = [1, 2, 5, 6, np.inf]
    names = ['ws', 'gov', 'seinc', 'seuninc']
    df.loc[:, 'cow'] = pd.cut(df['ljcw'],
                                 bins=bins, labels=names, 
                                 right=False, include_lowest=True)
    
    return df

def get_pt_var(df):
    df = df.copy()
    df['pt'] = 0
    df.loc[df.hrcheck == 1, 'pt'] = 1
    
    return df

def get_disability_var(df):
    df = df.copy()
    df['disability'] = 0
    df.loc[df.prdisflg == 1, 'disability'] = 1
    
    return df

def get_married_var(df):
    df = df.copy()
    df['married'] = 0
    df.loc[df.a_maritl.isin([1,2,3]), 'married'] = 1
    
    return df

def get_children_var(df):
    df = df.copy()
    df['children'] = 0
    df.loc[df.frelu18 > 0, 'children'] = 1
    
    return df

def get_log_income_var(df, cpi):
    df = df.copy()
    cpi = cpi.copy()
    
    # adjust incomes to price level
    df = df.merge(cpi[['year', 'pl_ratio']], on='year')
    df['income_adjusted'] = df.htotval * df.pl_ratio
    
    # initially set to 0
    df['log_income'] = 0.0
    # only assign log value to positive values - we will treat negative and 0 income as all having 0 here
    df.loc[df.income_adjusted > 0, 'log_income'] = \
        np.log(df[df.income_adjusted > 0].income_adjusted)
        
    # remove cpi column
    df = df.drop('pl_ratio', axis=1)
    
    return df

# This is the label for classification
def get_leaver_var(df):
    df = df.copy()
    df['leaver'] = 0
    df.loc[~df.pemlr.isin([1,2]), 'leaver'] = 1
    
    return df

asec = remove_rows(asec)
asec = get_edu_var(asec)
asec = get_race_var(asec)
asec = get_hispanic_var(asec)
asec = get_male_var(asec)
asec = get_citizenship_var(asec)
asec = get_cow_var(asec)
asec = get_pt_var(asec)
asec = get_disability_var(asec)
asec = get_married_var(asec)
asec = get_children_var(asec)
asec = get_log_income_var(asec, cpi)
asec = get_leaver_var(asec)

print('Percentage workers who are classified as leavers', 
      str(round(asec.leaver.sum()/len(asec), 4)*100) + '%')

Just over 5% of workers in our dataset are classified as "Labor force leavers"

### Final steps for preparing model data


In [ ]:
asec = asec.rename(columns={
    'a_dtocc': 'occ',
    'wemind': 'ind',
    'a_age': 'age'
})

model_data = asec[[
    'marsupwt', 'leaver',  # weight and label
    'age', 'log_income',  # continuous variables
    'pt', 'hisp', 'male', 'disability', 'married', 'children',  # binary variables
    'cow', 'race', 'citshp', 'edu', 'occ', 'ind' # categorical variables
]].copy()

cat_vars = ['cow', 'race', 'citshp', 'edu', 'occ', 'ind']

for var in cat_vars:
    model_data[var] = model_data[var].astype('category')

# one-hot encode categorical variables
model_data = pd.get_dummies(model_data, drop_first=True,
                            columns=cat_vars, dtype=np.int8)

## Models

### Split data into training and test sets


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# I'll use a smaller sample of the data to speed up processing times
sample = model_data.sample(200000, random_state=42).reset_index(drop=True)

# scale age and income variables
continuous_features = ['age', 'log_income']
scaler = StandardScaler()
sample[continuous_features] = scaler.fit_transform(sample[continuous_features])

X = sample.drop(columns=['leaver', 'marsupwt'])
y = sample['leaver']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('training size:', len(X_train))
print('test size:', len(X_test))

### PCA

Let's see if we can identify any patterns with Principal component Analysis.


In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Run PCA, reducing the data to 2 components
pca = PCA()
pca.fit(X_train)

explained_variance = np.cumsum(pca.explained_variance_ratio_)

# Plot the first two principal components
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(explained_variance) + 1),
        explained_variance, marker='o', linestyle='--')
plt.xlabel("Number of principale components")
plt.ylabel("cumulative explained variance")
plt.title("Explained variance by principal components")
plt.grid()
plt.show()

These are the variables assigned which explain the most variance in our PCA model. Perhaps unsurprisingly, the continuous variables (age and income) are the most explanatory:


In [ ]:
explained_variance_ratio = pca.explained_variance_ratio_

# Rank the importance of variables
importance_rank = np.argsort(explained_variance_ratio)[::-1]

# Print the ranked variables
pca_importance = []
for i, var_index in enumerate(importance_rank[:10]):
    pca_importance.append(X.columns[var_index])
    print(f"Rank {i+1}: Variable {X.columns[var_index]}")

### Random Forest

Let's implement a random forest model to gather more insights about our data.

Note that our data is imbalanced. We should use balanced class weights to mitigate this:


In [ ]:
leavers_frac = y.sum()/len(y)
leavers_frac_pct = round(leavers_frac*100, 1)

print(f"{leavers_frac_pct}% of the rows in our sample have a leaver value of 1")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(n_estimators=100,
                            class_weight='balanced',
                            random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Even when using the balanced class weights, our model predicts a low amount of leavers:


In [ ]:
pred_leavers_frac = y_pred.sum()/len(y_pred)
pred_leavers_frac_pct = round(pred_leavers_frac*100, 1)

print(f"{pred_leavers_frac_pct}% of training data was predicted to be leavers by the model")

Let's make a confusion matrix and compute some metrics:

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score

cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(cm).plot()

print('Recall:', recall_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('F1:', f1_score(y_test, y_pred))

Precision is quite high, meaning that the vast majority of positive predictions were correct. Recall was just over 50%, which means the model was only able to identify classify a little over half of the leavers. Given the imbalance in the data, this isn't terrible

Let's see which features this random forest assessed as most important. Just like we saw in PCA, income and age are the most important features, but this time income is ranked higher than age. Unlike what we saw in PCA, an industry and 5 occupations were among the most important features in this model.


In [ ]:
print(pd.Series(rf.feature_importances_, index=X.columns) \
    .sort_values(ascending=False)[:10])

Let's start adding these importance rankings to a DataFrame we can use to compare the rankings from different models:


In [ ]:
importance_rankings = pd.DataFrame({
    'Rank': list(range(1,11)),
    'PCA': pca_importance,
    'Random Forest': list(
        pd.Series(rf.feature_importances_, index=X.columns) \
        .sort_values(ascending=False)[:10].index),
})
importance_rankings